<a href="https://colab.research.google.com/github/balszeg/Thesis/blob/main/YOLOv4_BDD100K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Building DarkNet

Important to set the Colab runtime type to **GPU**.

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make darknet
!make

In [ ]:
# mount Google Drive + creat symbolic link to short its path
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive

## Dataset preprocessing

In [ ]:
%cd /content/darknet
!mkdir /content/darknet/data/bdd100k/

In [ ]:
# copy the dataset from drive
!cp /mydrive/bdd100k/bdd100k_images.zip /content/darknet/data/bdd100k/
!cp /mydrive/bdd100k/bdd100k_labels_release.zip /content/darknet/data/bdd100k/

In [ ]:
%cd /content/darknet/data/bdd100k/
!ls

In [ ]:
!unzip bdd100k_images.zip
!unzip bdd100k_labels_release.zip

In [ ]:
# delete the original zip files, to free up storage space
%cd /content/darknet/data/bdd100k
!find . -name "*.zip" -type f -delete

In [ ]:
# copy the necessary Python script and folder creating
%cd /content/darknet/data/bdd100k/bdd100k
!cp /mydrive/bdd100k/label_to_txt.py /content/darknet/data/bdd100k/bdd100k
!mkdir /content/darknet/data/bdd100k/bdd100k/labels/train
!mkdir /content/darknet/data/bdd100k/bdd100k/labels/val

In [ ]:
# convert the training label's .json to YOLO specific .txt
!python label_to_txt.py -l '/content/darknet/data/bdd100k/bdd100k/labels/bdd100k_labels_images_train.json' -d '/content/darknet/data/bdd100k/bdd100k/labels/train/'

In [ ]:
# check it
!ls /content/darknet/data/bdd100k/bdd100k/labels/train

In [ ]:
# convert the validation label's .json to YOLO specific .txt
!python label_to_txt.py -l '/content/darknet/data/bdd100k/bdd100k/labels/bdd100k_labels_images_val.json' -d '/content/darknet/data/bdd100k/bdd100k/labels/val/'

In [ ]:
# check it
!ls /content/darknet/data/bdd100k/bdd100k/labels/val

In [ ]:
# delete the original labels to free up storage space
%cd /content/darknet/data/bdd100k/bdd100k/labels
!find . -name "*.json" -type f -delete

## Training configuration

In [ ]:
# get the necessary config file from drive
# the parameter selection detailed in the thesis
%cd /content/darknet/cfg
!cp /content/gdrive/MyDrive/bdd100k/yolov4-bdd100k.cfg /content/darknet/cfg
!ls /content/darknet/cfg/

In [ ]:
# get the necessary .names and .data files from drive
!cp /content/gdrive/MyDrive/bdd100k/obj.names /content/darknet/data
!cp /content/gdrive/MyDrive/bdd100k/obj.data  /content/darknet/data

In [ ]:
# get the data path generator Python scripts from drive
!cp /mydrive/bdd100k/generate_train.py /content/darknet/data/bdd100k/
!cp /mydrive/bdd100k/generate_val.py /content/darknet/data/bdd100k/

In [ ]:
# excute them
%cd /content/darknet/data/bdd100k/
!python generate_train.py
!python generate_val.py

In [ ]:
# copy the generated .txt inside the DarkNet folder
!cp /content/darknet/data/bdd100k/bdd100k/images/100k/train.txt /content/darknet/data
!cp /content/darknet/data/bdd100k/bdd100k/images/100k/val.txt /content/darknet/data

In [ ]:
# copy the labels .txt to the same folders as the images
# the YOLO demands this, so it could access to data
!cp -a "/content/darknet/data/bdd100k/bdd100k/labels/train/." "/content/darknet/data/bdd100k/bdd100k/images/100k/train/"
!cp -a "/content/darknet/data/bdd100k/bdd100k/labels/val/." "/content/darknet/data/bdd100k/bdd100k/images/100k/val/"

In [ ]:
# check the training folder content
!ls '/content/darknet/data/bdd100k/bdd100k/images/100k/train/'

In [ ]:
# check the validation folder content
!ls '/content/darknet/data/bdd100k/bdd100k/images/100k/val/'

## Training

Pressing **Ctrl+Shift+I** (or **F12**) the browser console opens up. Copying this script is prevent the Colab's timeout.
```
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)
```


In [ ]:
%cd /content/darknet

In [ ]:
# training from scratch
!./darknet detector train data/obj.data cfg/yolov4-bdd100k.cfg -dont_show -map

In [ ]:
# training with pretrained weights
!./darknet detector train data/obj.data cfg/yolov4-bdd100k.cfg /mydrive/bdd100k/backup/yolov4-bdd100k_last.weights -dont_show -map

## Evaluation

In [ ]:
# calculate metrics
!./darknet detector map data/obj.data cfg/yolov4-bdd100k.cfg /mydrive/bdd100k/backup/yolov4-bdd100k_last.weights

In [ ]:
# testing on video
!./darknet detector demo data/obj.data cfg/yolov4-bdd100k.cfg yolov4_bdd100k_last.weights -dont_show /mydrive/video/traffic_video.mp4 -i 0 -out_filename /mydrive/video/results.avi